# 02. Pandas 데이터 전처리 - 실습 문제

## 실습 안내
- 총 10개 문제
- 각 문제는 실제 스마트팩토리 데이터 정제 시나리오
- 힌트를 참고하여 직접 코드 작성
- 결과를 확인하며 학습

## 데이터 로드

In [140]:
import pandas as pd
import numpy as np

# 데이터 불러오기
production_df = pd.read_csv('../data/05_production.csv', encoding='utf-8-sig')
quality_df = pd.read_csv('../data/07_quality_inspection.csv', encoding='utf-8-sig', na_values=['\\N', 'NULL', ''])
sensor_df = pd.read_csv('../data/08_sensor_data.csv', encoding='utf-8-sig')
operation_df = pd.read_csv('../data/06_equipment_operation.csv', encoding='utf-8-sig')

print("데이터 로드 완료!")
print(f"생산: {len(production_df):,}건")
print(f"품질: {len(quality_df):,}건")
print(f"센서: {len(sensor_df):,}건")
print(f"설비운영: {len(operation_df):,}건")

데이터 로드 완료!
생산: 1,872건
품질: 37,417건
센서: 10,920건
설비운영: 3,304건


---
## 문제 1: 결측치 비율 분석

**시나리오**: 품질검사 데이터의 각 컬럼별 결측치 비율을 파악하여 데이터 품질을 평가하세요.

**요구사항**:
1. 각 컬럼의 결측치 개수와 비율(%) 계산
2. 결측치가 있는 컬럼만 출력
3. 결측치 비율이 높은 순서로 정렬



In [141]:
# 1. 각 컬럼의 결측치 개수와 비율(%) 계산
pd.concat([quality_df.isna().sum(), (quality_df.isna().sum()/len(quality_df)*100).round(2)], axis=1)

,0,1
inspection_id,0,0.00
production_id,0,0.00
equipment_id,0,0.00
product_code,0,0.00
inspection_time,0,0.00
inspection_type,0,0.00
result,0,0.00
defect_code,16541,44.21
measurement_value,0,0.00
measurement_unit,0,0.00


In [142]:
# 2. 결측치가 있는 컬럼만 출력
missing_values = quality_df.isna().sum()
missing_values[missing_values > 0]

defect_code    16541
notes          37417
dtype: int64

In [143]:
# 3. 결측치 비율이 높은 순서로 정렬
data = [(quality_df['defect_code'].isna().sum()/len(quality_df)*100).round(2),
        (quality_df['notes'].isna().sum()/len(quality_df)*100).round(2)]
index = ['defect_code_miss_rate', 'notes_miss_rate']
df_1 = pd.Series(data=data, index=index)
df_1.sort_values(ascending=False)

notes_miss_rate          100.00
defect_code_miss_rate     44.21
dtype: float64

---
## 문제 2: 센서 데이터 결측치 보간

**시나리오**: 센서 데이터에 결측치가 발생했습니다. 시계열 특성을 고려하여 적절히 처리하세요.

**요구사항**:
1. INJ-001 설비의 2024-01-01 00:00:00 ~ 10:00:00 데이터 필터링
2. temperature 컬럼의 5개 값을 임의로 NaN으로 변경 (인덱스 5, 15, 25, 35, 45)
3. 선형 보간법(interpolate)으로 결측치 채우기
4. 보간 전후 비교 출력

**힌트**: `interpolate(method='linear')`

In [144]:
# 1. INJ-001 설비의 2024-01-01 00:00:00 ~ 10:00:00 데이터 필터링
sensor_df['measurement_time'] = pd.to_datetime(sensor_df['measurement_time'])
sensor_df[(sensor_df['equipment_id'] == 'INJ-001') &
          (sensor_df['measurement_time'].between('2024-01-01 00:00:00', '2024-01-01 10:00:00'))].reset_index()

,index,sensor_id,equipment_id,measurement_time,temperature,pressure,vibration,current,voltage,rpm,created_at
0,0,1,INJ-001,2024-01-01 00:00:00,183.93,148.65,2.6838,48.05,218.83,1795.32,2026-01-30 00:45:52
1,5,6,INJ-001,2024-01-01 01:00:00,179.26,155.23,2.6619,41.47,221.62,1792.30,2026-01-30 00:45:52
2,10,11,INJ-001,2024-01-01 02:00:00,179.78,146.19,2.3795,42.07,221.48,1805.22,2026-01-30 00:45:52
3,15,16,INJ-001,2024-01-01 03:00:00,181.30,143.83,2.4134,47.61,211.96,1803.69,2026-01-30 00:45:52
4,20,21,INJ-001,2024-01-01 04:00:00,179.96,152.57,2.4701,44.85,215.77,1769.70,2026-01-30 00:45:52
5,25,26,INJ-001,2024-01-01 05:00:00,176.14,149.33,2.5944,43.70,230.72,1812.68,2026-01-30 00:45:52
6,30,31,INJ-001,2024-01-01 06:00:00,181.93,152.47,2.3188,44.23,220.57,1813.24,2026-01-30 00:45:52
7,35,36,INJ-001,2024-01-01 07:00:00,181.88,156.25,2.2091,42.57,225.79,1815.83,2026-01-30 00:45:52
8,40,41,INJ-001,2024-01-01 08:00:00,184.31,147.68,2.5049,46.00,227.26,1819.19,2026-01-30 00:45:52
9,45,46,INJ-001,2024-01-01 09:00:00,181.25,145.22,2.3801,45.01,220.23,1791.00,2026-01-30 00:45:52


In [138]:
# 2. temperature 컬럼의 5개 값을 임의로 NaN으로 변경 (인덱스 5, 15, 25, 35, 45)
sensor_df.loc[5:45:10, 'temperature'] = pd.NA

In [65]:
# 3. 선형 보간법(interpolate)으로 결측치 채우기
sensor_df['temperature'].interpolate(method='linear')

0        183.93
1        173.06
2         82.29
3         87.62
4         20.78
          ...  
10915    192.48
10916    175.82
10917     84.57
10918     87.53
10919     29.81
Name: temperature, Length: 10920, dtype: float64

In [69]:
# 4. 보간 전후 비교 출력
pd.concat([sensor_df.loc[:46, 'temperature'], sensor_df['temperature'].interpolate(method='linear')[:47]], axis=1)

,temperature,temperature
0,183.93,183.930
1,173.06,173.060
2,82.29,82.290
3,87.62,87.620
4,20.78,20.780
5,NaN,96.500
6,172.22,172.220
7,89.25,89.250
8,82.54,82.540
9,24.48,24.480


---
## 문제 3: 중복 작업지시서 찾기

**시나리오**: 생산 데이터에서 같은 작업지시서 번호(work_order_no)로 중복 기록된 건을 찾으세요.

**요구사항**:
1. work_order_no 기준으로 중복 확인
2. 중복된 데이터만 추출하여 work_order_no로 정렬
3. 중복 건수 및 상위 10개 출력



In [76]:
# 1. work_order_no 기준으로 중복 확인
production_df.duplicated(subset=['work_order_no']).sum()

np.int64(2)

In [86]:
# 2. 중복된 데이터만 추출하여 work_order_no로 정렬
production_df[production_df.duplicated(subset=['work_order_no'])].sort_values('work_order_no')

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,cycle_time,work_order_no,lot_no,operator_id,shift,created_at,updated_at
1726,1727,PRESS-002,DASH-C,2024-03-24,2024-03-24 09:13:00,2024-03-24 11:07:26,91,87,73,14,78.92,WO202403248166,LOT2024032400201,OP002,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48
1851,1852,PRESS-002,DASH-C,2024-03-30,2024-03-31 06:58:00,2024-03-31 09:03:09,87,83,68,15,90.48,WO202403302194,LOT2024033000212,OP005,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48


In [ ]:
# 3. 중복 건수 및 상위 10개 출력


---
## 문제 4: 날짜/시간 변환 및 추출

**시나리오**: 설비운영 데이터의 시간 정보를 분석 가능한 형태로 변환하세요.

**요구사항**:
1. start_time, end_time을 datetime 타입으로 변환
2. 다음 컬럼 생성:
   - operation_date: 날짜만 추출
   - start_hour: 시작 시간(시)
   - duration_minutes: 운영 시간(분)
3. 처음 10개 행 출력



In [90]:
# 1. start_time, end_time을 datetime 타입으로 변환
operation_df['start_time'] = pd.to_datetime(operation_df['start_time'])
operation_df['end_time'] = pd.to_datetime(operation_df['end_time'])
operation_df

,operation_id,equipment_id,start_time,end_time,operation_status,stop_reason,stop_category,operator_id,notes,created_at
0,1,INJ-001,2024-01-01 08:00:00,2024-01-01 09:44:04,RUNNING,NaN,NaN,OP003,NaN,2026-01-30 00:43:52
1,2,INJ-001,2024-01-01 09:47:04,2024-01-01 10:27:25,IDLE,휴식,계획정지,OP001,NaN,2026-01-30 00:43:52
2,3,INJ-001,2024-01-01 10:28:25,2024-01-01 12:10:18,RUNNING,NaN,NaN,OP002,NaN,2026-01-30 00:43:52
3,4,INJ-001,2024-01-01 12:20:18,2024-01-01 15:29:11,RUNNING,NaN,NaN,OP002,NaN,2026-01-30 00:43:52
4,5,INJ-001,2024-01-01 15:38:11,2024-01-01 18:24:15,RUNNING,NaN,NaN,OP010,NaN,2026-01-30 00:43:52
...,...,...,...,...,...,...,...,...,...,...
3299,3300,ASM-001,2024-03-31 13:09:58,2024-03-31 14:36:50,RUNNING,NaN,NaN,OP010,NaN,2026-01-30 00:43:53
3300,3301,ASM-001,2024-03-31 14:41:50,2024-03-31 16:48:21,RUNNING,NaN,NaN,OP003,NaN,2026-01-30 00:43:53
3301,3302,ASM-001,2024-03-31 16:49:21,2024-03-31 20:10:41,RUNNING,NaN,NaN,OP002,NaN,2026-01-30 00:43:53
3302,3303,ASM-001,2024-03-31 20:17:41,2024-03-31 20:32:13,IDLE,자재 대기,자재대기,OP005,NaN,2026-01-30 00:43:53


In [121]:
# 2. 다음 컬럼 생성:
#    - operation_date: 날짜만 추출
#    - start_hour: 시작 시간(시)
#    - duration_minutes: 운영 시간(분)
operation_df['operation_date'] = operation_df['start_time'].dt.date
operation_df['start_hour'] = operation_df['start_time'].dt.hour
operation_df['duration_minutes'] = ((operation_df['end_time'] - operation_df['start_time']).dt.total_seconds()/60).round(2)

In [122]:
# 3. 처음 10개 행 출력
operation_df.head(10)

,operation_id,equipment_id,start_time,end_time,operation_status,stop_reason,stop_category,operator_id,notes,created_at,operation_date,start_hour,duration_minutes
0,1,INJ-001,2024-01-01 08:00:00,2024-01-01 09:44:04,RUNNING,NaN,NaN,OP003,NaN,2026-01-30 00:43:52,2024-01-01,8,104.07
1,2,INJ-001,2024-01-01 09:47:04,2024-01-01 10:27:25,IDLE,휴식,계획정지,OP001,NaN,2026-01-30 00:43:52,2024-01-01,9,40.35
2,3,INJ-001,2024-01-01 10:28:25,2024-01-01 12:10:18,RUNNING,NaN,NaN,OP002,NaN,2026-01-30 00:43:52,2024-01-01,10,101.88
3,4,INJ-001,2024-01-01 12:20:18,2024-01-01 15:29:11,RUNNING,NaN,NaN,OP002,NaN,2026-01-30 00:43:52,2024-01-01,12,188.88
4,5,INJ-001,2024-01-01 15:38:11,2024-01-01 18:24:15,RUNNING,NaN,NaN,OP010,NaN,2026-01-30 00:43:52,2024-01-01,15,166.07
5,6,INJ-001,2024-01-01 18:25:15,2024-01-01 19:36:56,BREAKDOWN,전기 이상,고장정지,OP009,NaN,2026-01-30 00:43:52,2024-01-01,18,71.68
6,7,INJ-002,2024-01-01 08:00:00,2024-01-01 09:17:24,RUNNING,NaN,NaN,OP010,NaN,2026-01-30 00:43:52,2024-01-01,8,77.40
7,8,INJ-002,2024-01-01 09:23:24,2024-01-01 12:34:45,RUNNING,NaN,NaN,OP001,NaN,2026-01-30 00:43:52,2024-01-01,9,191.35
8,9,INJ-002,2024-01-01 12:43:45,2024-01-01 13:57:56,RUNNING,NaN,NaN,OP010,NaN,2026-01-30 00:43:52,2024-01-01,12,74.18
9,10,INJ-002,2024-01-01 14:02:56,2024-01-01 14:48:45,IDLE,전공정 대기,기타,OP002,NaN,2026-01-30 00:43:52,2024-01-01,14,45.82


---
## 문제 5: 생산 효율 지표 계산

**시나리오**: 생산 데이터에서 주요 효율 지표를 계산하여 새 컬럼으로 추가하세요.

**요구사항**:
다음 컬럼 생성:
1. `defect_rate`: 불량률 (불량수 / 실제생산량 * 100, 소수점 2자리)
2. `achievement_rate`: 목표달성률 (실제생산량 / 목표생산량 * 100, 소수점 2자리)
3. `good_rate`: 양품률 (양품수 / 실제생산량 * 100, 소수점 2자리)
4. 통계 요약 출력 (describe)



In [110]:
# 1. `defect_rate`: 불량률 (불량수 / 실제생산량 * 100, 소수점 2자리)
production_df['defect_rate'] = round(production_df['defect_quantity']/production_df['actual_quantity']*100, 2)

In [112]:
# 2. `achievement_rate`: 목표달성률 (실제생산량 / 목표생산량 * 100, 소수점 2자리)
production_df['achievement_rate'] = round(production_df['actual_quantity']/production_df['target_quantity']*100, 2)

In [114]:
# 3. `good_rate`: 양품률 (양품수 / 실제생산량 * 100, 소수점 2자리)
production_df['good_rate'] = round(production_df['good_quantity']/production_df['actual_quantity']*100, 2)

In [116]:
# 4. 통계 요약 출력 (describe)
production_df.describe()

,production_id,target_quantity,actual_quantity,good_quantity,defect_quantity,cycle_time,defect_rate,achievement_rate,good_rate
count,1872.00000,1872.000000,1872.000000,1872.000000,1872.000000,1872.000000,1872.000000,1872.000000,1872.000000
mean,936.50000,115.255876,110.365385,99.213675,11.151709,76.635625,10.222692,95.729679,89.777308
std,540.54417,20.755408,21.582998,20.848651,5.910472,11.930266,5.141065,6.690435,5.141065
min,1.00000,80.000000,65.000000,55.000000,2.000000,48.270000,2.000000,71.530000,80.000000
25%,468.75000,97.000000,93.000000,82.000000,6.000000,67.657500,5.300000,94.530000,85.000000
50%,936.50000,115.500000,110.000000,98.000000,10.000000,75.505000,11.000000,95.800000,89.000000
75%,1404.25000,134.000000,128.000000,115.000000,16.000000,83.792500,15.000000,97.920000,94.700000
max,1872.00000,150.000000,157.000000,152.000000,26.000000,120.840000,20.000000,105.000000,98.000000


---
## 문제 6: 품질 등급 분류 (조건부 컬럼)

**시나리오**: 불량률에 따라 생산 품질을 4등급으로 분류하세요.

**요구사항**:
1. `quality_grade` 컬럼 생성 (불량률 기준):
   - 3% 미만: 'S등급'
   - 3~5% 미만: 'A등급'
   - 5~10% 미만: 'B등급'
   - 10% 이상: 'C등급'
2. 등급별 분포 출력
3. 각 등급의 평균 불량률 계산

**힌트**: `np.select()`, 조건 리스트와 선택 리스트

In [118]:
# 1. `quality_grade` 컬럼 생성 (불량률 기준):
#    - 3% 미만: 'S등급'
#    - 3~5% 미만: 'A등급'
#    - 5~10% 미만: 'B등급'
#    - 10% 이상: 'C등급'

def get_grade(rate):
    if rate < 3: return 'S등급'
    elif rate < 5: return 'A등급'
    elif rate < 10: return 'B등급'
    else: return 'C등급'

production_df['defect_rate'].apply(get_grade)

0       A등급
1       B등급
2       S등급
3       S등급
4       B등급
       ... 
1867    C등급
1868    C등급
1869    C등급
1870    C등급
1871    C등급
Name: defect_rate, Length: 1872, dtype: object

In [ ]:
# 2. 등급별 분포 출력
production_df[;de]

In [ ]:
# 3. 각 등급의 평균 불량률 계산

---
## 문제 7: 시간대 분류

**시나리오**: 생산 시작 시간을 기준으로 시간대를 분류하여 시간대별 생산 패턴을 분석하세요.

**요구사항**:
1. start_time을 datetime으로 변환
2. `time_shift` 컬럼 생성 (시작 시간 기준):
   - 06:00~14:00: '주간'
   - 14:00~22:00: '야간'
   - 22:00~06:00: '심야'
3. 시간대별 생산 건수 및 평균 불량률 계산

**힌트**: ` `np.select()` 또는 사용자 정의 함수 + `apply()`

In [ ]:
# 여기에 코드 작성


---
## 문제 8: 생산량 구간 분류

**시나리오**: 실제 생산량을 구간별로 나누어 생산 규모를 분류하세요.

**요구사항**:
1. `pd.cut()`을 사용하여 `production_scale` 컬럼 생성:
   - 0~50: '소량'
   - 50~100: '중량'
   - 100~150: '대량'
   - 150 이상: '초대량'
2. 구간별 생산 건수 출력
3. 각 구간의 평균 불량률 계산

**힌트**: `pd.cut()`, bins, labels

In [ ]:
# 여기에 코드 작성


---
## 문제 9: 설비 가동률 계산

**시나리오**: 설비운영 데이터에서 설비별 가동률을 계산하세요.

**요구사항**:
1. start_time, end_time을 datetime으로 변환
2. `operation_hours` 컬럼 생성 (운영 시간을 시간 단위로 변환)
3. operation_status가 'RUNNING'인 경우만 필터링
4. 설비별 총 가동시간 계산 및 정렬
5. 상위 5개 설비 출력

**힌트**: datetime 변환, 시간 차이 계산, `total_seconds()`

In [ ]:
# 여기에 코드 작성


---
## 문제 10: 종합 데이터 전처리 파이프라인

**시나리오**: 품질검사 데이터를 분석 가능한 형태로 완전히 전처리하세요.

**요구사항**:
1. 결측치 처리:
   - defect_code의 NaN을 'NONE'으로 변경
   - notes의 NaN을 빈 문자열로 변경
2. 날짜/시간 변환:
   - inspection_time을 datetime으로 변환
   - inspection_date, inspection_hour 컬럼 생성
3. 새 컬럼 생성:
   - `is_defect`: 불량 여부 (FAIL=1, PASS=0)
   - `measure_status`: 측정값 상태 분류
     * BUMPER-A 기준: 295~305 '정상', 290~295 또는 305~310 '주의', 나머지 '이상'
4. 최종 데이터 형태 확인 (shape, info, head)

**힌트**: 순차적으로 처리, 사용자 정의 함수, `apply()`

In [ ]:
# 여기에 코드 작성


---
## 수고하셨습니다!

### 학습 체크리스트
- [ ] 결측치 확인 및 처리 방법 이해
- [ ] 중복 데이터 찾기 및 제거
- [ ] 데이터 타입 변환 (특히 날짜/시간)
- [ ] 날짜/시간 요소 추출 및 연산
- [ ] 계산 컬럼 생성
- [ ] 조건부 컬럼 생성 (np.where, np.select)
- [ ] 범주형 변수 생성 (cut, qcut)
- [ ] 종합 데이터 전처리 파이프라인 구성

